<span style='color:red'> NOTE: You can only pass the lab, when you provide both code and markdown </span>

Use Code for your analysis
Use Markdown to document and elaborate on your findings, conclusions, assertions, etc.

# DS_ML_I_P1: Dataset creation from raw data 
Provided is a list of Excel-Files that stem from a radar measurement using an array of 15 Antennas and a frequency sweep. Another Excel sheet provides information on the type of object that should be detected and its orientation.

The overall task is to load the data into **a single dataframe**, add the **proper information on object**, **orientation** and **the name of the image that shows the object** (which is not provided here)


## 1. Load the data and check proper loading
Load all the data into a single dataframe so that
* The name of the file is a separat column
* Only the first five columns and all rows per Sheet tab should be integrated (15 Tabs in total, one per Antenna)
* Sheet tab name should be the major index in a multiindex column dataframe
* Tab column names should be the minor index
* After this dataframe has been created the object information, orientation and image name should be added as separate columns by integrating the information from the specific excel sheet.

In [48]:
import pandas as pd
import glob
import os
import numpy as np
import plotly.express as px
import plotly.io as pio

pio.templates.default = 'plotly_dark'


In [49]:
pd.options.display.max_columns=1000
pd.options.display.max_rows=50


### 1.1 Load Measurement Data 
Measurement data from multiple excel files are loaded, each containing measurement data from 15 antennas. 

The final dataframe is structured with antennas name as the major/1stlevel index column and the five measurement values as the 2nd level index column. 

A separate column `filename` is added for each row for getting the additional information later.


In [50]:
%%time
file_paths = glob.glob("P1b/Measurements_8_April_2023_IMP-SIMO/*.xls")

dfs =[]

for file in file_paths:
    file_data = []
    sheets = pd.read_excel(file, sheet_name=None, usecols=[0,1,2,3,4])

    for sheet_name, df_temp in sheets.items():
        multi_columns = [
            np.repeat(sheet_name,len(df_temp.columns)),
            df_temp.columns.to_list()
        ]
        # df_temp.columns = pd.MultiIndex.from_arrays(multi_columns, names=['major','minor'])
        df_temp.columns = pd.MultiIndex.from_arrays(multi_columns)

        file_data.append(df_temp)

    df_temp["filename"] = os.path.splitext(os.path.basename(file))[0]
    dfs.append(pd.concat(file_data, axis=1))

concatenated_data = pd.concat(dfs, ignore_index=True)

CPU times: total: 6.67 s
Wall time: 6.7 s


Concatenated data is then copied to a new dataframe

In [51]:
df_m = concatenated_data

### 1.2 Load Protocol Data
The measurement protocol excel files are loaded into one dataframe and each column name is translated accordingly to english

In [52]:
df_p = pd.read_excel("P1b/Messprotokoll_18_04_2023_open_V1.xlsx", skiprows=6, usecols="C:H").rename(columns={
    "Messung": "measurement", 
    "Gegenstand": "object", 
    "Postion": "position", 
    "Dateienname ": "filename", 
    "Bild ": "image",
    "Anmerkungen": "comments"
})

Set filename as dataframe's index for the later merge operation

In [53]:
df_p['filename'] = df_p['filename'].astype(str).str.strip()
df_p = df_p.set_index('filename')

### 1.3 Add Additional Information to Measurement Dataframe
The measurement dataframe is enriched with the additional information like the object information, position and image name as separate columns, by mapping the column filename to the relevant information in the protocol dataframe `df_p`.

In [54]:
df_m['filename'] = df_m['filename'].astype(str).str.strip()

df_m['object'] = df_m['filename'].map(df_p['object'])
df_m['position'] = df_m['filename'].map(df_p['position'])
df_m['image'] = df_m['filename'].map(df_p['image'])

In [55]:
df = df_m

## 2. Print some statistics and analyze

### 2.1 .info()
* A total of 3.000 rows across 79 columns were loaded. 
* There are 76 columns with the value type numeric (float64 and int64): all the measured values from each antenna (i.e. Magnitude, Phase, DAC Values, and Frequency), as well as image.
* There are 3 columns with categorical value type object: filename, object, and position.
* 3 columns contain missing values: object, position, and images.

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 79 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   (ANT 1, DAC Value)       3000 non-null   float64
 1   (ANT 1, DAC Value RAW)   3000 non-null   int64  
 2   (ANT 1, Magnitute)       3000 non-null   float64
 3   (ANT 1, Phase)           3000 non-null   float64
 4   (ANT 1, Frequency)       3000 non-null   int64  
 5   (ANT 2, DAC Value)       3000 non-null   float64
 6   (ANT 2, DAC Value RAW)   3000 non-null   int64  
 7   (ANT 2, Magnitute)       3000 non-null   float64
 8   (ANT 2, Phase)           3000 non-null   float64
 9   (ANT 2, Frequency)       3000 non-null   int64  
 10  (ANT 3, DAC Value)       3000 non-null   float64
 11  (ANT 3, DAC Value RAW)   3000 non-null   int64  
 12  (ANT 3, Magnitute)       3000 non-null   float64
 13  (ANT 3, Phase)           3000 non-null   float64
 14  (ANT 3, Frequency)      

There are in total 420 rows with missing values. That make up 14% of the total entries.

In [57]:
rows_with_missing_values = df[df.isna().any(axis=1)]
print(f"{len(rows_with_missing_values)} rows has missing values")
print(f"{round(len(rows_with_missing_values)/len(df)*100,0)} % of total entries")

420 rows has missing values
14.0 % of total entries


### 2.2 .describe()
To be found below is the detail of each column of the dataset:
* As it can be seen from the description, the `Frequency` of all antennas are all equals in every rows and every columns with the frequency 2.450000e+09 for min,max,and 0 standard deviation
* DAC and DAC Value RAW all have the same average, min, as well as max value accross antenna's

In [58]:
df.describe()

ANT 1                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -29.440857   -11.037441  2.450000e+09   
std       0.727482    346.419787    21.308478   100.516179  0.000000e+00   
min      -1.576100    800.000000   -76.132400  -179.746000  2.450000e+09   
25%      -0.956600   1095.000000   -49.023000   -98.424450  2.450000e+09   
50%      -0.337100   1390.000000   -34.079850   -24.726700  2.450000e+09   
75%       0.282400   1685.000000    -5.197550    71.956325  2.450000e+09   
max       0.901900   1980.000000     2.967040   179.945000  2.450000e+09   

             ANT 2                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -28.584241    -8.454822  2.450000e+09   
std       0.727482    346.419787    20.660701   101.662437  0.000000e+00   
min      -1.576100    800.000000   -78.798000  -179.855000  2.450000e+09   
25%      -0.956600   1095.000000   -47.094225  -104.927500  2.450000e+09   
50%      -0.337100   1390.000000   -33.717600    -0.022706  2.450000e+09   
75%       0.282400   1685.000000    -5.459223    80.736000  2.450000e+09   
max       0.901900   1980.000000     2.600070   179.882000  2.450000e+09   

             ANT 3                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -28.496339     6.162484  2.450000e+09   
std       0.727482    346.419787    19.896935   105.454240  0.000000e+00   
min      -1.576100    800.000000   -65.850600  -179.841000  2.450000e+09   
25%      -0.956600   1095.000000   -46.240275   -93.691275  2.450000e+09   
50%      -0.337100   1390.000000   -33.768350    20.151350  2.450000e+09   
75%       0.282400   1685.000000    -6.185160    95.470150  2.450000e+09   
max       0.901900   1980.000000     1.951980   179.968000  2.450000e+09   

             ANT 4                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -27.625137    -5.295042  2.450000e+09   
std       0.727482    346.419787    20.533735   102.950631  0.000000e+00   
min      -1.576100    800.000000   -66.213900  -179.830000  2.450000e+09   
25%      -0.956600   1095.000000   -47.068525  -108.048500  2.450000e+09   
50%      -0.337100   1390.000000   -32.086750     0.879465  2.450000e+09   
75%       0.282400   1685.000000    -4.311635    87.005050  2.450000e+09   
max       0.901900   1980.000000     3.010070   179.962000  2.450000e+09   

             ANT 5                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -28.421452   -14.991958  2.450000e+09   
std       0.727482    346.419787    20.865768   101.811997  0.000000e+00   
min      -1.576100    800.000000   -72.225700  -179.865000  2.450000e+09   
25%      -0.956600   1095.000000   -48.134000  -103.428500  2.450000e+09   
50%      -0.337100   1390.000000   -31.816800   -35.516350  2.450000e+09   
75%       0.282400   1685.000000    -4.999288    70.339400  2.450000e+09   
max       0.901900   1980.000000     2.312730   179.475000  2.450000e+09   

             ANT 6                                                        \
         DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.0000

This is then later checked and confirmed below, that every value in every `Frequency` column is constant, 2450000000.

In [59]:
freq_columns = df.xs('Frequency', axis=1, level=1)
print(f"Unique values of the frequency columns: {pd.unique(freq_columns.values.ravel())}")

Unique values of the frequency columns: [2450000000]


The following code checks whether all DAC Value RAW, as well as the DAC Value, on every antennas are equals (exactly the same), by counting unique values per row and lastly returning the set of counts found. If the result is {1}, it means that for every row, all DAC Value RAW and (not RAW) values across columns are identical.

In [60]:
dac_raw_columns = df.xs('DAC Value RAW', axis=1, level=1)
dac_raw_columns.nunique(axis=1).unique()

array([1])

In [61]:
dac_columns = df.xs('DAC Value', axis=1, level=1)
dac_columns.nunique(axis=1).unique()

array([1])

The code below examines the three columns with missing values (object, position, and image) and retrieves their unique values.

It was found that these entries are either unlabeled (i.e., no object), or the objects are labeled as 'Mit Kamera' and 'Ohne Kamera', hence no detail about its position. The unlabeled entries could potentially represent values needed for testing or evaluating a trained model, while the 'Mit Kamera' and 'Ohne Kamera' labels are additional entries that were not fully described in the measurement report PDF.

Furthermore, not every object was measured at the same set of angles. For instance, some open-end wrenches were only measured at certain angles, while others were measured at a completely different set of angles.

In [62]:
filtered_df = df[[('object', ''), ('position', ''), ('image', ''), ('filename', '')]].copy()
filtered_df.columns = filtered_df.columns.droplevel(level=1)
filtered_df.set_index('object').drop_duplicates().sort_values(by=['object', 'position'])

,position,image,filename
object,,,
Maulschlüssel 14/15,0°,29.0,1804202330
Maulschlüssel 14/15,lang1,7.0,1804202308
Maulschlüssel 14/16,30°,30.0,1804202331
Maulschlüssel 14/16,lang2,8.0,1804202309
Maulschlüssel 14/17,45°,31.0,1804202332
Maulschlüssel 14/17,lang3,9.0,1804202310
Maulschlüssel 14/18,60°,32.0,1804202333
Maulschlüssel 14/18,quer1,10.0,1804202311
Maulschlüssel 14/19,90°,33.0,1804202334


## 3. Visualize the data
* Scatter Plot
* Box Plot
* Histogram

### 3.1 Scatter Plot

Two scatter plots of phase and magnitude values measured on Antenna 8 are shown below. Antenna 8 is chosen since it is centrally positioned. 

The data is categorized by object type and measurement angle.

In [63]:
fig = px.scatter(df, x=df[('ANT 8','Magnitute')], y=df[('ANT 8','Phase')], color=df[('object','')], height=800, title='Relationship Between Phase and Magnitude on Antenna 8 for Different Objects')
fig.show()

fig = px.scatter(df, x=df[('ANT 8','Magnitute')], y=df[('ANT 8','Phase')], color=df[('position','')], height=800, title='Relationship Between Phase and Magnitude on Antenna 8 for Different Position(Angle)')
fig.show()

### 3.2 Box Plot

Show two boxplots to visualize the distribution of Magnitude and Phase accross ddifferent antennas.

The result:
* There is a uniform magnitude distribution across all 15 antennas.
* The distribution of phase values vary slightly across antennas, and their medians appear to follow a kind of pattern within antenna groups 1–5, 6–10, and 11–15

In [65]:
df_melted = df.xs('Magnitute', axis=1, level=1).melt(var_name='Column', value_name='Value')
fig = px.box(df_melted, x='Column', y='Value', title="Magnitude")
fig.show()

df_melted = df.xs('Phase', axis=1, level=1).melt(var_name='Column', value_name='Value')
fig = px.box(df_melted, x='Column', y='Value', title="Phase")
fig.show()

### 3.3 Histogram
3.3.1 Show the Phase value distribution from ANT 5 and ANT 6 antennas, as they are among the antennas that show slightly noticeable variation in distribution according to the previous boxplots

In [66]:
fig = px.histogram(df, x=df[('ANT 5','Phase')], title='Phase of Antenna 5')
fig.show()
fig = px.histogram(df, x=df[('ANT 6','Phase')], title='Phase of Antenna 6')
fig.show()


## 4. Conclusion


1) It was found that some measurement entries are either unlabeled (i.e., no object assigned), or the objects are labeled as 'Mit Kamera' and 'Ohne Kamera', which lack details about their position. The unlabeled entries could potentially represent values needed for testing or evaluating a trained model, while the 'Mit Kamera' and 'Ohne Kamera' labels are additional entries not fully described in the measurement report PDF. These entries make up 14% of the total rows, as indicated by the NaN values in the object/position/image columns. These could possibly be unlabeled entries that might be needed to test or evaluate a trained model.

2) Not every object was measured at the same set of angles. For instance, some open-end wrenches were only measured at specific angles (0 degrees, horizontal 1), while others were measured at a completely different set of angles. In general, there are more (and different) objects than those specified in the measurement report PDF.

3) The columns DAC Value, DAC Value RAW, and Frequency contain identical values across rows for different antennas. Therefore, the relevant measurement values are the Phase and Magnitude. By visualizing their relationships with different types of graphs:
    * The scatter plot of phase and magnitude values shows that certain objects or angles follow distinct patterns.
    * Boxplots demonstrate unified distributions of magnitude values across 15 antennas, with a slightly varying distribution of phase values.
    * Histograms were used to show the distribution in detail. The phase values of antennas 5 and 6 were compared, and it can be seen that the distribution varies between these two antennas. Hence, the finding from the boxplots is confirmed.